In [ ]:
m!pip install neural_tangents

In [ ]:
!export XLA_FLAGS=--xla_gpu_strict_conv_algorithm_picker=false


In [ ]:
import os
os.environ['XLA_FLAGS'] = '--xla_gpu_strict_conv_algorithm_picker=false'

# Now, import JAX and other libraries
import jax


Firstly, I wrote a CNN with jax and train it on mnist with SGD

---



In [ ]:
import jax
from jax import grad, jit
from jax.example_libraries import stax, optimizers
from jax.example_libraries.stax import Conv, Relu, Flatten, Dense
from tensorflow.keras.datasets import mnist
import jax
#CNN structure
#the cov kernel width is 512, which I believe can be a limit to inifnite on the dataset of mnist
init_fn, apply_fn = stax.serial(
    Conv(512, (3, 3), padding='SAME'), Relu,
    Flatten,
    Dense(10)
)
# Step 2: Initialize parameters
key = jax.random.PRNGKey(0)
input_shape = (-1, 28, 28, 1)
output_shape, params = init_fn(key, input_shape)
#load mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((-1, 28, 28, 1)) / 255.0
test_images = test_images.reshape((-1, 28, 28, 1)) / 255.0


def update(params, x, y, opt_state):
    """Single SGD update step."""
    value, grads = jax.value_and_grad(cross_entropy_loss)(params, x, y)
    opt_state = opt_update(0, grads, opt_state)
    return get_params(opt_state), opt_state, value

def cross_entropy_loss(params, x, y):
    logits = apply_fn(params, x)
    return -jnp.mean(jnp.sum(jax.nn.log_softmax(logits) * jax.nn.one_hot(y,10), axis=1))  # Cross-entropy loss

def accuracy(params, x, y):
    preds = apply_fn(params, x)
    return jnp.mean(jnp.argmax(preds, axis=1) == jnp.argmax(y, axis=1))

# Step 5: Training
batch_size = 64
num_epochs = 5
step_size = 0.01
opt_init, opt_update, get_params = optimizers.sgd(step_size)
opt_state = opt_init(params)

import matplotlib.pyplot as plt
loss_ls = []
acc_ls = []
print('start: ')
for epoch in range(num_epochs):
    for i in range(0, 25600, batch_size):
        batch_images = train_images[i:i+batch_size]
        batch_labels = train_labels[i:i+batch_size]
        params, opt_state, loss = update(params, batch_images, batch_labels, opt_state)
        loss_ls.append(loss)
        if i % 12800 == 0:
            print(f'Epoch {epoch + 1}, Batch Index {i}, Loss: {loss}')
    train_acc = accuracy(params, train_images, train_labels)
    acc_ls.append(train_acc)
    print(f'Epoch {epoch + 1}, Loss: {loss}, Training Accuracy: {train_acc}')

plt.plot(epoch, loss_ls)
plt.xscale('log')
plt.xlabel('epoch')
plt.ylabel('Loss')
plt.title('Training')
plt.show()

plt.plot(epoch, acc_ls)
plt.xscale('log')
plt.xlabel('epoch')
plt.ylabel('ACC')
plt.title('Training')
plt.show()


test_acc = accuracy(params, test_images, test_labels)
print(f'Test Accuracy: {test_acc}')

11490434/11490434 [==============================] - 1s 0us/step
start: 


KeyboardInterrupt: ignored

comment: i think it definitely converge, but due to the limit of my hardware and space, I can't run fully the experiment. Therefore, let's go to the next step for ntk.

In [ ]:
import jax
import jax.numpy as jnp
from jax import random
from neural_tangents import stax
from neural_tangents import predict

from jax import grad, jit
from jax.example_libraries import stax, optimizers
from jax.example_libraries.stax import Conv, Relu, Flatten, Dense
from tensorflow.keras.datasets import mnist


#still the smae network
init_fn, apply_fn = stax.serial(
    Conv(512, (3, 3), padding='SAME'), Relu,
    Flatten,
    Dense(10)
)

import tensorflow_datasets as tfds

def load_mnist():
    ds_builder = tfds.builder('mnist')
    ds_builder.download_and_prepare()
    train_ds = tfds.as_numpy(ds_builder.as_dataset(split=tfds.Split.TRAIN, batch_size=-1))
    test_ds = tfds.as_numpy(ds_builder.as_dataset(split=tfds.Split.TEST, batch_size=-1))
    return train_ds['image'], train_ds['label'], test_ds['image'], test_ds['label']

train_images, train_labels, test_images, test_labels = load_mnist()

train_images = jnp.array(train_images.reshape((-1, 28, 28, 1)) / 255.0, dtype=jnp.float32)
test_images = jnp.array(test_images.reshape((-1, 28, 28, 1)) / 255.0, dtype=jnp.float32)
print(train_images.shape)

(60000, 28, 28, 1)


In [ ]:
import neural_tangents
from neural_tangents import empirical_kernel_fn
from jax import random
import time

key = random.PRNGKey(0)
_, params = init_fn(key, input_shape=(-1, 28, 28, 1))

n = 10000
slicing = jnp.arange(n)
train_images = train_images[slicing]
train_labels = train_labels[slicing]
test_images = test_images[slicing]
test_labels = test_labels[slicing]
print(train_images.shape)
print(test_images.shape)
# Construct the kernel function
kernel_fn = empirical_kernel_fn(apply_fn)
#kernel regression

BS = 32
# Perform NTK regression
ntk = kernel_fn(train_images, train_images, 'ntk', params)
ntk = neural_tangents.batch(ntk, device_count = 0, batch_size= BS)
start = time.time()
predict_fn = predict.gradient_descent_mse(ntk, train_labels, learning_rate=0.1)

train_predictions = predict_fn(t=None, k_test_train = ntk)

"""
print("Shape of train_predictions:", train_predictions.shape)

if train_predictions.ndim == 2:
    train_predicted_labels = jnp.argmax(train_predictions, axis=1)
    train_accuracy = jnp.mean(train_predicted_labels == train_labels)
    print(f"Training accuracy: {train_accuracy * 100:.2f}%")
else:
    print("Unexpected shape for train_predictions")

train_predicted_labels = jnp.argmax(train_predictions, axis=1)

 train_accuracy= jnp.mean(train_predicted_labels == train_labels)
print(f"Training accuracy (subset): {train_accuracy * 100:.2f}%")
"""
print(train_predictions)
train_accuracy= jnp.mean(train_predictions == train_labels)
print(f"Training accuracy (subset): {train_accuracy * 100:.2f}%")
test_predictions = predict_fn(t=None, k_test_train=kernel_fn(test_images, train_images, 'ntk', params))

# Convert predictions to labels
print(test_predictions)
accuracy = jnp.mean(test_predictions == test_labels)
print(f"Test accuracy: {accuracy * 100:.2f}%")


(10, 28, 28, 1)
(10, 28, 28, 1)
[4. 1. 0. 7. 8. 1. 2. 7. 1. 6.]
Training accuracy (subset): 100.00%
[4. 1. 0. 7. 8. 1. 2. 7. 1. 6.]
Test accuracy: 0.00%


successful implement of ntk of a cnn

In [ ]:
import time
from absl import app
import jax.numpy as jnp
import neural_tangents as nt
from neural_tangents import stax
from examples import datasets
from examples import util

_TRAIN_SIZE = 9856
_TEST_SIZE = 9856
_BATCH_SIZE = 64
# Build data pipelines.
print('Loading data.')
x_train, y_train, x_test, y_test = datasets.get_dataset('cifar10',
                                                          _TRAIN_SIZE,
                                                          _TEST_SIZE)

x_train = jnp.array(x_train.reshape((-1, 32, 32, 3)) / 255.0, dtype=jnp.float32)
x_test = jnp.array(x_test.reshape((-1, 32, 32, 3)) / 255.0, dtype=jnp.float32)


# Build the infinite network.
_, _, kernel_fn = stax.serial(
    stax.Conv(512, (3, 3), padding='SAME'),
    stax.Relu(),
    stax.Flatten(),
    stax.Dense(20)
)

# Optionally, compute the kernel in batches, in parallel.
kernel_fn = nt.batch(kernel_fn,
                     device_count=0,
                     batch_size=_BATCH_SIZE)

print(kernel_fn.shape)

start = time.time()
# Bayesian and infinite-time gradient descent inference with infinite network.
predict_fn = nt.predict.gradient_descent_mse_ensemble(kernel_fn, x_train,
                                                        y_train, diag_reg=1e-3)
_, fx_test_ntk = predict_fn(x_test=x_test)

print(fx_test_ntk.shape, fx_test_ntk[1,:])

fx_test_ntk.block_until_ready()


duration = time.time() - start
print('Kernel construction and inference done in %s seconds.' % duration)

duration = time.time() - start
print('Kernel construction and inference done in %s seconds.' % duration)

# Print out accuracy and loss for infinite network predictions.
loss = lambda fx, y_hat: 0.5 * jnp.mean((fx - y_hat) ** 2)
#util.print_summary('NNGP test', y_test, fx_test_nngp, None, loss)
util.print_summary('NTK test', y_test, fx_test_ntk, None, loss)


Loading data.
(9856, 10) [ 0.3149414   0.14334106  0.04386139  0.27424622 -0.03878784  0.14709473
 -0.04695129 -0.03023148  0.08893967  0.28730774]
Kernel construction and inference done in 30.077336311340332 seconds.
Kernel construction and inference done in 30.077478408813477 seconds.

Evaluating Network on NTK test data.
---------------------------------------
Network Accuracy = 0.4844764471054077
Network Loss = 0.04205967113375664
---------------------------------------
